In [219]:
import docxtpl as tpl
import matplotlib.pyplot as plt
import pandas as pd
import json
from datetime import datetime, timedelta
import xml.etree.cElementTree as ET
from typing import Union
from pathlib import Path
import win32com.client as win32
import pythoncom
import shutil
import os


In [246]:
def load_data(fund_dir) -> dict:
    file = f"data/{fund_dir}/data.json"
    data = json.load(open(file))
    return data

def load_returns(fund_dir) -> pd.DataFrame:
    file = f"data/{fund_dir}/returns.csv"
    returns = pd.read_csv(file, index_col=0, parse_dates=True)
    return returns

def create_returns_chart(fund_dir) -> str:
    """
    Creates a cumulative returns chart for the fund and benchmark. Returns the path to the saved chart.

    Args:
        fund_dir: The directory of the fund.

    Returns:
        The path to the saved chart.
    """
    fig, ax = plt.subplots(figsize=(6, 4))
    
    data = load_data(fund_dir)
    returns = pd.read_csv(f"data/{fund_dir}/returns.csv", index_col=0, parse_dates=True)

    returns.dropna(inplace=True)    
    bmk = returns[data['benchmarkName']]
    etf = returns[data['fundName']]

    (bmk.cumsum()*100).plot(ax=ax, label=data['benchmarkName'], color=data['theme']['accent2'], linewidth=2)
    (etf.cumsum()*100).plot(ax=ax, label=data['fundTicker'], color=data['theme']['accent1'], linewidth=2)
    
    ax.set_ylabel("Cumulative Return (%)")
    ax.legend()
    plt.tight_layout()
    plt.savefig(f"temp/{data['fundName']} returns.png")
    plt.close()
    return f"temp/{data['fundName']} returns.png"

def update_data(fund_dir, data: dict) -> dict:
    """
    Updates the data dictionary with the latest performance data. Returns the updated data dictionary.

    Args:
        fund_dir: The directory of the fund.
        data: The data dictionary to be updated.
    
    Returns:
        The updated data dictionary
    """
    returns = load_returns(fund_dir)
    etf: pd.Series = returns[data['fundName']]

    def get_periodic_returns(etf, years):
        if any(etf.iloc[-12*years:].isna()):
            return "N/A"
        perf = (etf.iloc[-12*years:]+ 1).prod()**(1/years) - 1
        return f"{perf*100:.2f}%"
    
    _1yr = get_periodic_returns(etf, 1)
    _3yr = get_periodic_returns(etf, 3)
    _5yr = get_periodic_returns(etf, 5)
    _10yr = get_periodic_returns(etf, 10)

    data['performanceData']['_1Year'] = _1yr
    data['performanceData']['_3Year'] = _3yr
    data['performanceData']['_5Year'] = _5yr
    data['performanceData']['_10Year'] = _10yr

    data['date'] = etf.index[-1].strftime("%B %d, %Y")
    data['inceptionDate'] = etf.index[0].strftime("%B %d, %Y")
    return data

def create_xml_theme_colors(
    filename: Union[str, Path],
    accent1: str='#FFFFFF',
    accent2: str='#FFFFFF',
    accent3: str='#FFFFFF',
    accent4: str='#FFFFFF',
    accent5: str='#FFFFFF',
    accent6: str='#FFFFFF',
    text1: str='#000000',
    text2: str='#000000',
    background1: str='#FFFFFF',
    background2: str='#FFFFFF',
    hyperlink: str='#0563C1',
    hyperlink_followed: str='#954F72',
):
    root = ET.Element("a:clrScheme")
    root.attrib['xmlns:a'] = "http://schemas.openxmlformats.org/drawingml/2006/main"
    root.attrib['name'] = "test"

    # text 1
    ET.SubElement(
        ET.SubElement(root, "a:dk1"), 
        "a:sysClr", val="windowText", lastClr=text1.replace('#', '')
    )
    
    # background 1
    ET.SubElement(
        ET.SubElement(root, "a:lt1"), 
        "a:sysClr", val="window", lastClr=background1.replace('#', '')
    )

    # text 2
    ET.SubElement(
        ET.SubElement(root, "a:dk2"), 
        "a:srgbClr", val=text2.replace('#', '')
    )

    # background 2
    ET.SubElement(
        ET.SubElement(root, "a:lt2"), 
        "a:srgbClr", val=background2.replace('#', '')
    )

    # accents
    for i, hex in enumerate([accent1, accent2, accent3, accent4, accent5, accent6]):
        ET.SubElement(
            ET.SubElement(root, f"a:accent{i+1}"), 
            "a:srgbClr", val=hex.replace('#', '')
        )

    # hyperlinks
    ET.SubElement(
        ET.SubElement(root, "a:hlink"), 
        "a:srgbClr", val=hyperlink.replace('#', '')
    )
    ET.SubElement(
        ET.SubElement(root, "a:folHlink"), 
        "a:srgbClr", val=hyperlink_followed.replace('#', '')
    )

    tree = ET.ElementTree(root)
    tree.write(filename, encoding='utf-8', xml_declaration=True)

def modify_theme_colors(doc_file, theme_file):
    app = win32.DispatchEx("Word.Application", pythoncom.CoInitialize())
    app.Visible = False
    doc = app.Documents.Open(doc_file)
    doc.DocumentTheme.ThemeColorScheme.Load(theme_file) 
    doc.SaveAs(doc_file)
    app.Quit()

def save_docx_to_pdf(docx_file, pdf_file):
    app = win32.DispatchEx("Word.Application", pythoncom.CoInitialize())
    app.Visible = False
    doc = app.Documents.Open(docx_file)
    doc.SaveAs(pdf_file, FileFormat=17)
    doc.Close()
    app.Quit()

def render_template(fund_dir):
    data = load_data(fund_dir)
    data = update_data(fund_dir, data)
    returns_chart = create_returns_chart(fund_dir)

    template = tpl.DocxTemplate(f"Factsheet Template.docx")
    template.render(data, autoescape=True) # render the data dictionary

    # replace the pictures in the template
    template.replace_pic("returnsChart", returns_chart)
    template.replace_pic("logo", f"data/{fund_dir}/logo.png")

    # save the rendered docx file
    out_file = f"output/{data['fundName']} Factsheet {data['date']}.docx"
    template.save(out_file) 

    create_xml_theme_colors("temp/theme.xml", **data['theme'])
    modify_theme_colors(os.getcwd() + '\\' + out_file, os.getcwd() + '\\temp\\theme.xml')
    save_docx_to_pdf(os.getcwd() + '\\' + out_file, os.getcwd() + f'\\output\\{data["fundName"]} Factsheet {data["date"]}.pdf')


In [247]:
dirs = os.listdir('data')
for fund_dir in dirs:
    render_template(fund_dir)
    print(f"Created factsheet for {fund_dir}")

Created factsheet for esg_etf
Created factsheet for spy_etf
